# 공통 프롬프트 테스트

In [ ]:
# grouped_pages.json 불러오기
import json

json_file_path = '/Users/yoon/BOAZ_ADV/Wang_Gyu/20240424_data_v2/grouped_pages.json'


# 파일 읽어서 변수에 저장
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [12]:
from openai import OpenAI
import json
from tqdm import tqdm
import os
from dotenv import load_dotenv

system_prompt = """
# Role and Objective
You are a specialized medical expert in Pediatric Anesthesia.
Your primary task is to generate high-quality Korean Q&A data from provided medical guidelines for training a medical question-answering model.

# Core Principles
- Maintain strict medical accuracy and precision
- Focus on practical clinical scenarios
- Ensure questions are specific and actionable
- Include relevant numerical values when available
- Generate questions only from provided context
- Use clear and professional Korean language

# Question Generation Guidelines
1. Content Adherence:
   - Generate questions strictly based on provided content
   - Include specific numerical values (doses, weights, ages, etc.)
   - Focus on practical clinical applications

2. Question Format:
   - Use open-ended, short-answer format
   - Write questions in Korean
   - Make questions specific and clinically relevant
   - Avoid multiple-choice formats

3. Reference Requirements:
   - Include all supporting sentences
   - For tables: include entire table content
   - For figures: include surrounding text and captions

# Answer Guidelines
- Start reasoning directly with a quote (no phrases like '문서에 따르면', '문서에 의하면')
- Start with a step-by-step reasoning using quotes from the context
- Enclose all direct quotes in **##begin_quote## ... ##end_quote##**
- End with `<ANSWER>: ...` in Korean (full sentence) 

# Output Format
Return your output in this **strict JSON format**:
[
  {
    "question": "한국어로 된 의학 질문",
    "reference_sentences": [
      "관련 문장 1",
      "관련 문장 2"
    ],
    "answer": "##Reason: ...\n<ANSWER>: ..."
  }
]

# Quality Controls
- Ensure medical accuracy
- Verify all numerical values
- Check Korean language usage
- Validate JSON format
- Confirm reference alignment

# Error Prevention
- Double-check medical terminology
- Verify numerical calculations
- Ensure proper Korean grammar
- Validate JSON structure
- Confirm context alignment

# Example Outputs
Return as a **valid JSON array** of objects. Each object must look like:

[
  {{
    "question": "5살 23kg 소아 환자의 마취 중 적절한 수액 주입량은?",
    "reference_sentences": [
      "20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다.",
      "이 수액량은 전신마취 중 적절한 수분 공급을 위해 사용된다."
    ],
    "answer": "##Reason: 문서의 ##begin_quote## 20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다 ##end_quote## 라는 설명에 따르면, 23kg 소아는 1500mL + (3×20mL) = 1560mL가 필요합니다.\n<ANSWER>: 5살 23kg 소아의 마취 중 유지 수액량은 1560mL입니다."
  }},
  {{
    "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떤 처치를 해야 하나요?",
    "reference_sentences": [
      "Laryngospasm이 의심되는 경우 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다.",
      "신속한 인식과 처치가 저산소증을 예방하는 데 중요하다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다 ##end_quote## 라고 되어 있습니다. 이는 환자의 기도를 유지하고 저산소증을 방지하는 데 중요한 조치입니다.\n<ANSWER>: Laryngospasm이 의심되는 경우 jaw thrust, 양압 환기, succinylcholine 투여를 포함한 즉각적인 처치가 필요합니다."
  }}
]
"""

In [13]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 결과 저장용 딕셔너리
results = {}

# 페이지별 질문 생성
for page_number, context in enumerate(data['28']):
    
    user_prompt = f'# Input Data Structure {{"context": "{context}", "num_questions per page": 4}}'

    try:
        response = client.responses.create(
            model="gpt-4.1",  # 또는 "gpt-3.5-turbo"
            input=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5
        )
        output = response.output[0].content[0].text
        parsed = json.loads(output)
        results[page_number] = parsed

        break

    except Exception as e:
        print(f"Error on page {page_number}: {e}")

# 결과 출력
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "0": [
    {
      "question": "2세 소아에서 마취 유도 시 적절한 프로포폴 용량은 얼마인가요?",
      "reference_sentences": [
        "프로포폴의 소아 마취 유도 용량은 2~3mg/kg이다.",
        "소아는 성인에 비해 프로포폴에 대한 감수성이 높으므로 용량 조절이 필요하다."
      ],
      "answer": "##Reason: ##begin_quote## 프로포폴의 소아 마취 유도 용량은 2~3mg/kg이다 ##end_quote## 라는 문장에 따라 2세 소아의 프로포폴 적정 용량은 체중 1kg당 2~3mg입니다. 소아는 성인보다 감수성이 높으므로 주의가 필요합니다. <ANSWER>: 2세 소아의 마취 유도 시 프로포폴 용량은 2~3mg/kg입니다."
    },
    {
      "question": "소아에서 마취 시 저혈압이 발생하면 우선적으로 어떤 처치를 시행해야 하나요?",
      "reference_sentences": [
        "소아에서 마취 중 저혈압이 발생하면 우선적으로 수액을 투여하고, 필요시 혈관수축제를 사용한다.",
        "저혈압의 원인 평가와 신속한 처치가 중요하다."
      ],
      "answer": "##Reason: ##begin_quote## 소아에서 마취 중 저혈압이 발생하면 우선적으로 수액을 투여하고, 필요시 혈관수축제를 사용한다 ##end_quote## 라는 지침에 따라, 저혈압 발생 시에는 먼저 수액 투여를 시행해야 하며, 이후 혈관수축제 투여를 고려합니다. <ANSWER>: 소아 마취 중 저혈압이 발생하면 우선적으로 수액을 투여해야 합니다."
    },
    {
      "question": "4kg 신생아의 마취 유지에 필요한 수액량은 얼마인가요?",
      "reference_sentences": [
        "소아의 유지 수액량은 4-2-1 법칙을 적용한다.",
       

# 학술지 전처리

### 1) 폴더 안에 있는 20개의 pdf upstage 이용해서 전처리 후 json으로 저장

### 이후 전처리는 각자 해보고 금욜날 공유 ㄱㄱㄱ

In [1]:
from typing import TypedDict, Annotated, List, Dict
import operator

class GraphState(TypedDict):
    folderpath : Annotated[str, "filepath"]  # 원본 파일 경로
    
    analyzed_files : Annotated[List, "analyzed_files"]

    metadata: Annotated[List[Dict], operator.add]  # parsing metadata (api, model, usage)
    
    elements_from_parser: Annotated[List[Dict], "elements_from_parser"]  

state = GraphState(folderpath='/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지')
state

{'folderpath': '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지'}

In [ ]:
import requests
import json
import os

DEFAULT_CONFIG = {
    "ocr": True,
    "coordinates": True,
    "output_formats": "['html', 'text', 'markdown']",
    "model": "document-parse",
    "base64_encoding": "['figure', 'chart', 'table']",
}

def analyze_layout(state: GraphState):
    """
    분할된 PDF 파일들을 Upstage API로 전송하여 레이아웃 분석을 수행하고,
    결과 JSON 파일들을 저장하는 함수.

    Returns:
    - list: 성공적으로 분석된 파일들의 결과 JSON 파일 경로 목록.
    """
    docs_folder = '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지'    # <- 20개 pdf 파일이 있는 폴더 경로

    output_folder = "/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage"      # <- 최종 upstage 결과 저장 폴더
    
    api_url = "https://api.upstage.ai/v1/document-ai/document-parse"
    api_key = os.environ.get("UPSTAGE_API_KEY")

    # split_docs 폴더 내의 모든 PDF 파일 가져오기
    pdf_files = [f for f in os.listdir(docs_folder) if f.endswith(".pdf")]
    output_paths = []
    metadata_list = []
    
    for pdf_file in pdf_files:
        file_path = os.path.join(docs_folder, pdf_file)
        output_json_path = os.path.join(output_folder, f"{os.path.splitext(pdf_file)[0]}.json")
        print(f"📤 파일 업로드 중: {pdf_file}")
        
        with open(file_path, "rb") as pdf:
            response = requests.post(
                api_url,
                headers={"Authorization": f"Bearer {api_key}"},
                data=DEFAULT_CONFIG,
                files={"document": pdf}
            )
        
        # 응답 확인 및 JSON 파일 저장
        if response.status_code == 200:
            result = response.json()
            with open(output_json_path, "w", encoding="utf-8") as json_file:
                json.dump(result, json_file, ensure_ascii=False, indent=4)
            print(f"✅ 분석 결과 저장 완료: {output_json_path}")
            output_paths.append(output_json_path)
            meta = {
                "id": pdf_file,
                "model": result.get("model"),
                "usage": result.get("usage")
            }
            metadata_list.append(meta)
        else:
            print(f"❌ 오류 발생 ({response.status_code}): {response.text}")
    
    print("🎉 모든 파일 분석 완료!")
    return GraphState(metadata=metadata_list,
                      analyzed_files=output_paths)

In [16]:
state_out = analyze_layout(state)
state.update(state_out)
state

📤 파일 업로드 중: Apm006-03-19.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm006-03-19.json
📤 파일 업로드 중: Apm005-04-18.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm005-04-18.json
📤 파일 업로드 중: Apm005-04-19.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm005-04-19.json
📤 파일 업로드 중: Apm006-03-18.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm006-03-18.json
📤 파일 업로드 중: Apm006-03-20.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm006-03-20.json
📤 파일 업로드 중: Apm006-01-18.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm006-01-18.json
📤 파일 업로드 중: Apm005-03-17.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm005-03-17.json
📤 파일 업로드 중: Apm006-02-19.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm006-02-19.json
📤 파일 업로드 중: Apm005-02-16.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV/Wang_Gyu/학술지_upstage/Apm005-02-16.json
📤 파일 업로드 중: Apm006-02-21.pdf
✅ 분석 결과 저장 완료: /Users/yoon/BOAZ_ADV

{'folderpath': '/Users/yoon/BOAZ_ADV/Wang_Gyu/학술지',
 'metadata': [{'id': 'Apm006-03-19.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 4}},
  {'id': 'Apm005-04-18.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 4}},
  {'id': 'Apm005-04-19.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 4}},
  {'id': 'Apm006-03-18.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 6}},
  {'id': 'Apm006-03-20.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 4}},
  {'id': 'Apm006-01-18.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 4}},
  {'id': 'Apm005-03-17.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 5}},
  {'id': 'Apm006-02-19.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 4}},
  {'id': 'Apm005-02-16.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 3}},
  {'id': 'Apm006-02-21.pdf',
   'model': 'document-parse-250404',
   'usage': {'pages': 4}},
  {'id